# Building Network services



# Socket programmming

If you want to use a protocol that Python doesn't support natively (or just want to use your own protocol), you can always use the lower-level `socket` module in the standard library.

But first, a (brief) review of network protocol layers

![Image](data/img/OSI.png "OSI Stack")

## Basic socket programming

In [1]:
import socket

In [2]:
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock = socket.socket()

In [3]:
cat /etc/services

# Network services, Internet style
#
# Note that it is presently the policy of IANA to assign a single well-known
# port number for both TCP and UDP; hence, officially ports have two entries
# even if the protocol doesn't support UDP operations.
#
# Updated from https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.xhtml .
#
# New ports will be added on request if they have been officially assigned
# by IANA and used in the real-world or are needed by a debian package.
# If you need a huge list of used numbers please install the nmap package.

tcpmux		1/tcp				# TCP port service multiplexer
echo		7/tcp
echo		7/udp
discard		9/tcp		sink null
discard		9/udp		sink null
systat		11/tcp		users
daytime		13/tcp
daytime		13/udp
netstat		15/tcp
qotd		17/tcp		quote
chargen		19/tcp		ttytst source
chargen		19/udp		ttytst source
ftp-data	20/tcp
ftp		21/tcp
fsp		21/udp		fspd
ssh		22/tcp				# SSH Remote Login Protocol
telnet		23/tcp
smtp		25/tcp		mail
time		37/tcp		tims

In [4]:
sock.connect(('www.cnn.com', 80))   # /etc/services
http_req = '''GET / HTTP/1.1
Host: www.cnn.com
User-Agent: Advanced-Python/1.0
Accept: */*

'''

In [5]:
sock.sendall(http_req.encode('utf-8'))
response = sock.recv(1024)
sock.close()
print(len(response))

560


In [6]:
print(response.decode('utf8'))

HTTP/1.1 301 Moved Permanently
Server: Varnish
Retry-After: 0
Content-Length: 0
Cache-Control: public, max-age=600
Location: https://www.cnn.com/
Accept-Ranges: bytes
Date: Tue, 02 Nov 2021 20:07:59 GMT
Via: 1.1 varnish
Connection: close
Set-Cookie: countryCode=US; Domain=.cnn.com; Path=/; SameSite=Lax
Set-Cookie: stateCode=GA; Domain=.cnn.com; Path=/; SameSite=Lax
Set-Cookie: geoData=marietta|GA|30062|US|NA|-400|broadband|34.010|-84.450; Domain=.cnn.com; Path=/; SameSite=Lax
X-Served-By: cache-pdk17828-PDK
X-Cache: HIT
X-Cache-Hits: 0




### Socket programming basics:

#### Client

 - `connect()`
 - `send()`, `recv()`
 - generally does _not_ `bind()` (but may)
 
#### Server

 - `bind()` to a well-known port
 - `listen()` to set up a *connection backlog*
 - `accept()` incoming connections, returning **a new socket**

In [7]:
sock.listen?

In [8]:
from contextlib import closing

def echo_server(port):
    srv = socket.socket()
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind(('localhost', port))
    srv.listen(0)
    with closing(srv):
        print('Waiting for connections on localhost:{}'.format(port))
        peer_sock, peer_addr = srv.accept()
        print('got connection from {}'.format(peer_addr))
        with closing(peer_sock):
            buffer = peer_sock.recv(1000)
            print('Received "{}"'.format(buffer))
            peer_sock.sendall(buffer)

In [9]:
echo_server(8042)

Waiting for connections on localhost:8042
got connection from ('127.0.0.1', 45310)
Received "b'Here is some text\n'"


### Better: use a handler in a thread

In [10]:
import threading

def handle_echo(sock, addr):
    while True:
        buffer = sock.recv(1000)
        print('Received {}'.format(buffer))
        if not buffer:
            print('Socket closed, exiting thread')
            break
        sock.sendall(buffer)
        
def echo_server(port):
    srv = socket.socket()
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind(('localhost', port))
    srv.listen(0)
    with closing(srv):
        print('Waiting for connections on localhost:{}'.format(port))
        peer_sock, peer_addr = srv.accept()
        print('got connection from {}'.format(peer_addr))
        t = threading.Thread(target=handle_echo, args=(peer_sock, peer_addr))
        t.start()

In [11]:
echo_server(8042)

Waiting for connections on localhost:8042
got connection from ('127.0.0.1', 45316)
Received b'I\n'
Received b'should\n'
Received b'be\n'
Received b'able\n'
Received b'to\n'
Received b'echo\n'
Received b'lots\n'
Received b'of times\n'
Received b''
Socket closed, exiting thread


## Using SocketServer as a socket server framework

In [12]:
import socketserver

class MyEchoHandler(socketserver.BaseRequestHandler):
    def handle(self):
        while True:
            buffer = self.request.recv(1000)
            print('Received {}'.format(buffer))
            if not buffer:
                print('Socket disconnected, exiting handler')
                break
            self.request.sendall(buffer)

In [13]:
server = socketserver.TCPServer(('localhost', 8042), MyEchoHandler)

In [14]:
server.serve_forever()

Received b'Here is an echoing socket\n'
Received b'that works\n'
Received b''
Socket disconnected, exiting handler
Received b"And we see it doesn't\nhandle concurrency \nat all\n"
Received b''
Socket disconnected, exiting handler


KeyboardInterrupt: 

In [15]:
server = socketserver.ThreadingTCPServer(('localhost', 8043), MyEchoHandler)

In [16]:
server.serve_forever()

Received b'Echo?\n'
Received b'You bet!\n'
Received b'asdf\n'
Received b'asdfasdf\n'
Received b''
Socket disconnected, exiting handler
Received b''
Socket disconnected, exiting handler


KeyboardInterrupt: 

# UDP  - DNS example

from https://routley.io/posts/hand-writing-dns-messages/

In [17]:
sock = socket.socket(type=socket.SOCK_DGRAM)

In [18]:
msg = '''
AA AA 01 00 00 01 00 00 00 00 00 00
07 65 78 61 6d 70 6c 65 03 63 6f 6d 
00 00 01 00 01
'''

In [19]:
msg = ''.join(msg.split()).encode('utf-8')
msg

b'AAAA01000001000000000000076578616d706c6503636f6d0000010001'

In [20]:
import binascii
b_msg = binascii.unhexlify(msg)
b_msg

b'\xaa\xaa\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x07example\x03com\x00\x00\x01\x00\x01'

In [21]:
sock.sendto(b_msg, ('8.8.8.8', 53))
sock.recvfrom(4096)

(b'\xaa\xaa\x81\x80\x00\x01\x00\x01\x00\x00\x00\x00\x07example\x03com\x00\x00\x01\x00\x01\xc0\x0c\x00\x01\x00\x01\x00\x00A\xfe\x00\x04]\xb8\xd8"',
 ('8.8.8.8', 53))

In [25]:
for octet in 93,184,216,34:
    print(chr(octet), hex(octet), octet)

] 0x5d 93
¸ 0xb8 184
Ø 0xd8 216
" 0x22 34


In [23]:
!dig example.com


; <<>> DiG 9.16.1-Ubuntu <<>> example.com
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 20954
;; flags: qr rd ad; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 0
;; WARNING: recursion requested but not available

;; QUESTION SECTION:
;example.com.			IN	A

;; ANSWER SECTION:
example.com.		0	IN	A	93.184.216.34

;; Query time: 50 msec
;; SERVER: 172.28.112.1#53(172.28.112.1)
;; WHEN: Tue Nov 02 13:30:14 PDT 2021
;; MSG SIZE  rcvd: 56



If you _are_ building your own protocol layer, you'll probably want to become familiar with the `struct` module:

```python
import struct

struct.pack(...)
```

In [26]:
import struct

In [27]:
struct.pack('iii', 0x7aaa5555, 0x7aaa5555, 0x7aaa5555)

b'UU\xaazUU\xaazUU\xaaz'

In [28]:
struct.unpack('iii', _27)

(2057983317, 2057983317, 2057983317)

In [29]:
hex(2057983317)

'0x7aaa5555'

# Lab 

Open the [socket lab][socket-lab]

[socket-lab]: ./socket-lab.ipynb